# If running on Google Colab

You must start by installing both the chess and stockfish libraries.

In [ ]:
!pip install chess

Next, you need to give the script access to your Google Drive, or mount a different directory where your files are stored.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/deep-learning-su2024-chess-project/

# Import Libraries

In [1]:
import chess
import chess.pgn
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import OpeningAPIScraper

# Read in PGN file

Change these values to what suits your needs.

max_game_number = maximum number of games you will look at.

PGN_file = file path to the PGN you want to read including the file name.

In [ ]:
max_game_number = 100
PGN_file = 'lichess_db_standard_rated_2013-01.pgn'

In [ ]:
games = []
black_elo = []
white_elo = []
results = []
openings = []
with open(PGN_file) as f:
  game_number = 1
  while game_number < max_game_number:
    game = chess.pgn.read_game(f)
    if game is None:
      break
    moves = [str(m) for m in game.mainline_moves()]
    if len(moves) > 0:
      games.append(moves)
      black_elo.append(game.headers['BlackElo'])
      white_elo.append(game.headers['WhiteElo'])
      results.append(game.headers['Result'])
      openings.append(game.headers['Opening'])
    game_number += 1

# Reading Moves and Finding Openings

In [ ]:
# Read current CSV file to avoid duplicate openings
df = pd.read_csv('openings.csv', index_col=0)

In [ ]:
# Go through games to get the opening moves for each game
# Opening "ends" after 3 moves are made where no known opening is found
for game in games:
  board = chess.Board()
  i = 0
  for move in game:
    san = board.san(chess.Move.from_uci(move))
    board.push(chess.Move.from_uci(move))
    if board.fen() not in df['FEN'].values:
      try:
        bo, data = OpeningAPIScraper.fen_to_opening(board.fen())
      except AttributeError:
        time.sleep(3)
        bo, data = OpeningAPIScraper.fen_to_opening(board.fen())
    
    if bo == True:
      if data[0] not in df['Name'].values:
        df.loc[len(df)] = [board.fen(), data[0], data[1], move, san, data[2], data[3], data[4], data[5], data[6], data[7]]
      i = 0
    if bo == False:
      i += 1
      if i ==3:
        break

In [ ]:
df.to_csv('openings.csv')